In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from cabm import post_processing as ppr

In [ ]:
# Figure setup
# Set the font to Courier
mpl.rcParams['font.family'] = 'Courier'

# Set the line weight to thin
mpl.rcParams['lines.linewidth'] = 0.75

# Set the color scheme to black and white
mpl.rcParams['text.color'] = 'black'
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['ytick.color'] = 'black'
mpl.rcParams['axes.edgecolor'] = 'black'

In [ ]:
agent_df = pd.read_pickle('agent_output_ad_increment.pkl')

In [ ]:
agent_df = ppr.add_date_column(agent_df)

In [ ]:
def calculate_units(
    agent_df: pd.DataFrame,
    baseline_units_column: str = "Baseline_Units",
    units_to_purchase_column: str = "Units_to_Purchase",
    incremental_promo_units_column: str = "Incremental_Promo_Units",
    incremental_ad_units_column: str = "Incremental_Ad_Units",
    decremental_units_column: str = "Decremental_Units",
    brand_column: str = "Brand_Choice"
) -> pd.DataFrame:
    """
    Computes the sum of various unit columns for each brand purchased, grouped by Date and Step, and pivots the result.

    Parameters:
    agent_df (pd.DataFrame): The agent level data with 'Brand_Choice', 'Baseline_Units', 'Units_To_Purchase', 
                             'Incremental_Promo_Units', 'Incremental_Ad_Units', 'Decremental_Units', 'Date', and 'Step' columns.
    baseline_units_column (str): The column name for baseline units. Default is 'Baseline_Units'.
    units_to_purchase_column (str): The column name for units to purchase. Default is 'Units_To_Purchase'.
    incremental_promo_units_column (str): The column name for incremental promo units. Default is 'Incremental_Promo_Units'.
    incremental_ad_units_column (str): The column name for incremental ad units. Default is 'Incremental_Ad_Units'.
    decremental_units_column (str): The column name for decremental units. Default is 'Decremental_Units'.
    brand_column (str): The column name for brand choice. Default is 'Brand_Choice'.

    Returns:
    pd.DataFrame: A DataFrame with the sum of various unit columns for each brand, grouped by Date and Step, pivoted to have a column for each brand.
    """
    # Initialize a list to store the results
    units_data = []

    # Iterate over the DataFrame rows
    for (step, agent_id), row in agent_df.iterrows():
        brand = row[brand_column]
        date = row["Date"]
        baseline_units = row[baseline_units_column]
        units_to_purchase = row[units_to_purchase_column]
        incremental_promo_units = row[incremental_promo_units_column]
        incremental_ad_units = row[incremental_ad_units_column]
        decremental_units = row[decremental_units_column]

        units_data.append({
            "Brand": brand,
            "Date": date,
            "Step": step,
            "Baseline_Units": baseline_units,
            "Units_to_Purchase": units_to_purchase,
            "Incremental_Promo_Units": incremental_promo_units,
            "Incremental_Ad_Units": incremental_ad_units,
            "Decremental_Units": decremental_units
        })

    # Create a new DataFrame with the units data for each brand
    units_df = pd.DataFrame(units_data)

    # Group by 'Date', 'Step', and 'Brand' and sum the units
    grouped_units_df = (
        units_df.groupby(["Date", "Step", "Brand"])
        .sum()
        .reset_index()
    )

    # Pivot the DataFrame to have a column for each brand's units
    pivot_df = grouped_units_df.pivot_table(
        index=["Date", "Step"],
        columns="Brand",
        values=["Baseline_Units", "Units_to_Purchase", "Incremental_Promo_Units", "Incremental_Ad_Units", "Decremental_Units"],
        fill_value=0
    ).reset_index()

    # Flatten the column MultiIndex
    pivot_df.columns.name = None
    pivot_df.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in pivot_df.columns]

    return pivot_df

In [ ]:
all_units = calculate_units(agent_df)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_waterfall_for_brand(units_df: pd.DataFrame, brand: str, date: str):
    """
    Returns a Plotly Waterfall chart for a given brand and date.

    Parameters:
    units_df (pd.DataFrame): The DataFrame containing unit data.
    brand (str): The brand for which to plot the waterfall chart.
    date (str): The date for which to plot the waterfall chart.

    Returns:
    go.Figure: A Plotly Figure object representing the waterfall chart.
    """
    # Filter the DataFrame for the given date
    date_df = units_df[units_df['Date'] == date]

    if date_df.empty:
        print(f"No data available for the date: {date}")
        return None

    # Define the column names for the brand
    baseline_units_col = f"Baseline_Units_{brand}"
    incremental_promo_units_col = f"Incremental_Promo_Units_{brand}"
    incremental_ad_units_col = f"Incremental_Ad_Units_{brand}"
    decremental_units_col = f"Decremental_Units_{brand}"
    units_to_purchase_col = f"Units_to_Purchase_{brand}"

    # Check if the brand columns exist in the DataFrame
    if not all(col in date_df.columns for col in [baseline_units_col, incremental_promo_units_col, incremental_ad_units_col, decremental_units_col, units_to_purchase_col]):
        print(f"Data for brand {brand} is not available.")
        return None

    # Extract the values for the brand
    baseline_units = date_df[baseline_units_col].iloc[0]
    incremental_promo_units = date_df[incremental_promo_units_col].iloc[0]
    incremental_ad_units = date_df[incremental_ad_units_col].iloc[0]
    decremental_units = date_df[decremental_units_col].iloc[0]
    units_to_purchase = date_df[units_to_purchase_col].iloc[0]

    # Calculate the discrepancy
    total_units = baseline_units + incremental_promo_units + incremental_ad_units - decremental_units
    discrepancy = units_to_purchase - total_units

    # Create the waterfall chart
    fig = go.Figure(go.Waterfall(
        name=brand,
        orientation="v",
        measure=["relative", "relative", "relative", "relative", "relative", "total"],
        x=["Baseline Units", "Incremental Promo Units", "Incremental Ad Units", "Decremental Units (Elasticity)", "Pantry Overflow", "Units to Purchase"],
        y=[baseline_units, incremental_promo_units, incremental_ad_units, -decremental_units, discrepancy, units_to_purchase],
        connector={"line": {"color": "rgb(63, 63, 63)"}},
        increasing={"marker": {"color": "green"}},
        decreasing={"marker": {"color": "red"}},
        totals={"marker": {"color": "blue"}}
    ))

    fig.update_layout(
        title=f"Waterfall Chart for {brand} on {date}",
        waterfallgap=0.3
    )

    return fig

# Example usage
fig = plot_waterfall_for_brand(all_units, 'A', '2021-08-15')
if fig:
    fig.show()

In [ ]:
all_units.iloc[:,2:].sum()